# 04_01: Creating a GraphRAG pipeline with LangChain to query your data

In [ ]:
import os
from dotenv import load_dotenv

from langchain.prompts import PromptTemplate
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
from langchain_openai import ChatOpenAI

In [ ]:
load_dotenv(dotenv_path='../.env')

URI = os.getenv("NEO4J_URI")
USER = os.getenv("NEO4J_USER")
PWD = os.getenv("NEO4J_PWD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0, model_name="gpt-4o")
llm_transformer = LLMGraphTransformer(llm=llm)
graph = Neo4jGraph(url=URI, username=USER, password=PWD)

In [ ]:
graph.query('MATCH (n) RETURN COUNT(n) as TOTAL')

In [ ]:
CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to
answer questions about skiing.
Convert the user's question based on the schema.
When you are presented with query properties such as id's like "grass skiing",
be sure to convert the first letter to capital case, such as "Grass Skiing"
before you run the Cypher query.

Schema: {schema}
Question: {question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True,
    allow_dangerous_requests=True
)

In [ ]:
query_string = """
    Is the activity of cross-country skiing
    associated with the equipment of ski lifts?
"""

cypher_chain.invoke({"query": query_string})

In [28]:
cypher_chain.invoke({"query": "Tell me about ski touring."})



> Entering new GraphCypherQAChain chain...


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The semantics of using colon in the separation of alternative relationship types will change in a future version. (Please use ':PRACTICED_IN|USES|ASSOCIATED_WITH|REQUIRES|INCLUDES|SIMILAR_TO|EVOLVED_INTO|PART_OF|FORM_OF|COMPONENT|HAS_CONCEPT|HAS_LIST|HAS_TYPE|HAS_ORGANIZATION|BRANCHED_FROM|ALSO_KNOWN_AS|PRACTICED_AT|HELD_AT|PRACTICED_BY|AS_OF|PREDOMINANT_REGION|BIRTH|NAMED_AFTER|WITHOUT_USE_OF|USED_AS|PRACTICED_AS|NOT_ASSOCIATED_WITH|INVOLVES|PIONEER|OPTIONAL_EQUIPMENT|IS_A_VARIANT_OF|POPULAR_IN|RECOGNIZED_BY|EVOLVED_FROM|ADMINISTERED_BY|USED_FOR_TRANSPORT|POPULAR_LOCATION|LOCATED_IN|ORIGIN' instead)} {position: line: 2, column: 541, offset: 547} for query: 'cypher\nMATCH (a:Activity {id: "Ski Touring"})-[:PRACTICED_IN|:USES|:ASSOCIATED_WITH|:REQUIR

Generated Cypher:
cypher
MATCH (a:Activity {id: "Ski Touring"})-[:PRACTICED_IN|:USES|:ASSOCIATED_WITH|:REQUIRES|:INCLUDES|:SIMILAR_TO|:EVOLVED_INTO|:PART_OF|:FORM_OF|:COMPONENT|:HAS_CONCEPT|:HAS_LIST|:HAS_TYPE|:HAS_ORGANIZATION|:BRANCHED_FROM|:ALSO_KNOWN_AS|:PRACTICED_AT|:HELD_AT|:PRACTICED_BY|:AS_OF|:PREDOMINANT_REGION|:BIRTH|:NAMED_AFTER|:WITHOUT_USE_OF|:USED_AS|:PRACTICED_AS|:NOT_ASSOCIATED_WITH|:INVOLVES|:PIONEER|:OPTIONAL_EQUIPMENT|:IS_A_VARIANT_OF|:POPULAR_IN|:RECOGNIZED_BY|:EVOLVED_FROM|:ADMINISTERED_BY|:USED_FOR_TRANSPORT|:POPULAR_LOCATION|:LOCATED_IN|:ORIGIN]->(related)
RETURN a, related

Full Context:
[{'a': {'id': 'Ski Touring'}, 'related': {'id': 'Alpine Skiing Equipment'}}, {'a': {'id': 'Ski Touring'}, 'related': {'id': 'Nordic Ski Touring Equipment'}}, {'a': {'id': 'Ski Touring'}, 'related': {'id': 'Telemark Skiing Equipment'}}, {'a': {'id': 'Ski Touring'}, 'related': {'id': 'Alpine Touring Equipment'}}, {'a': {'id': 'Ski Touring'}, 'related': {'id': 'John "Snowshoe" Thom

{'query': 'Tell me about ski touring.',
 'result': 'Ski touring is related to Alpine Skiing Equipment, Nordic Ski Touring Equipment, Telemark Skiing Equipment, and Alpine Touring Equipment. Notable individuals associated with ski touring include John "Snowshoe" Thompson, Cecil Slingsby, Adolfo Kind, Arnold Lunn, Ottorino Mezzalama, and Patrick Vallençant.'}